In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import pickle



for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

    
def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df


def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df


def get_filtered_df(df, set_date, interval):
    # Convert set_date to the desired format
#     set_date = pd.to_datetime(set_date, format='%Y%m%d')
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    start_date = set_date - BDay(interval-1) 
    filtered_df = df[(df['日期'] >= start_date) & (df['日期'] <= set_date)]
    return filtered_df


def get_filtered_df_after(df, set_date, interval):
    # Convert set_date to the desired format
#     set_date = pd.to_datetime(set_date, format='%Y%m%d')
    set_date = pd.to_datetime(set_date)
    # Get the date that is 30 business days before set_date
    end_date = set_date + BDay(interval)
    filtered_df = df[(df['日期'] >= set_date) & (df['日期'] <= end_date)]
    return filtered_df

def get_today(df, set_date):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    filtered_df = df[df['日期'] == set_date]
    return filtered_df



def get_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['收盘涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth



def get_vol_growth(filtered_df):
    total_log_growth = filtered_df.groupby('股票代码')['成交量涨幅对数'].sum()
    total_growth = np.exp(total_log_growth) - 1
    return total_growth


def get_average_vol(filtered_df):
    average_vol = filtered_df.groupby('股票代码')['成交量'].mean()
    return average_vol

def get_average_price(filtered_df):
    average_price = filtered_df.groupby('股票代码')['收盘'].mean()
    return average_price

def get_average_volatility(filtered_df):
    average_volatility = filtered_df.groupby('股票代码')['振幅'].mean()
    return average_volatility

# 判断近期连续上涨
def whether_price_rise_in_last_n_days(filtered_df, lowwer, upper):
    # Group by 'Scode' and check if all differences are positive
    is_increasing = filtered_df.groupby('股票代码')['涨跌幅'].apply(lambda x: ((x >= lowwer) & (x < upper)).all())
    return is_increasing



# 判断近期成交量连续上涨
def whether_trade_vol_rise_in_last_n_days(filtered_df,extent):
    # Check if all percentage changes are within the range of 100% to 120%
    is_within_range = filtered_df.groupby('股票代码')['成交量变化百分比'].apply(lambda x: (x >= extent).all())
    return is_within_range

# 判断换手率大于n%
def whether_turnover_rate_larger_than(filtered_df,bottom):
    # Check if all percentage changes are within the range of 100% to 120%
    is_larger = filtered_df.groupby('股票代码')['换手率'].apply(lambda x: (x >= bottom).all())
    return is_larger

def get_n_days_latter(df,set_date,n_days):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    future_day = set_date + BDay(n_days)
    filtered_df = df[df['日期'] == future_day]
    return filtered_df

    
# last_30_day_growth_less_than_20p, total rise in a period
def period_price_condition(last_period, R_l, R_h):
    last_period_growth = get_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    last_period_growth_condition = list(last_period_growth_condition.index)
    set1 = set(last_period_growth_condition)
    return set1

# period_vol_condition
def period_vol_condition(last_period, R_l, R_h):
    last_period_growth = get_vol_growth(last_period)
    last_period_growth_condition = last_period_growth[(last_period_growth >= R_l) & (last_period_growth < R_h)]
    set1 = set(last_period_growth_condition.index)
    return set1

# price_rise_in_last_3_days, rise continiously
def recent_price_condition(new2_df,set_date,ndays,nday_lw,nday_up):
    # price rice in ndays1
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    price_rise_ndays = whether_price_rise_in_last_n_days(last_n_days,lowwer=nday_lw, upper=nday_up)
    price_rise_ndays = price_rise_ndays.loc[price_rise_ndays]
    price_rise_ndays = list(price_rise_ndays.index) # is a list
    set2 = set(price_rise_ndays)
    return set2


# volume_twice_times_of_montly_avg
def today_vol_condition(today, period, times):
    today_vol = get_average_vol(today)
    last_period_avg_vol = get_average_vol(period)
    vol_series = pd.concat([today_vol, last_period_avg_vol],axis=1).set_axis(['Series1', 'Series2'], axis=1)
    volume_times_of_period_avg = vol_series[vol_series['Series1'] > times*vol_series['Series2']]
    volume_times_of_period_avg = list(volume_times_of_period_avg.index)
    set3 = set(volume_times_of_period_avg)
    return set3

# price_rise_in_last_1_days
def today_price_condition(today,onelw,oneup):
    # price rice today
    price_rise_today = whether_price_rise_in_last_n_days(today,lowwer=onelw, upper=oneup)
    price_rise_today = price_rise_today.loc[price_rise_today]
    price_rise_today = list(price_rise_today.index) # is a list
    set4 = set(price_rise_today)
    return set4


def vol_rise_in_last_n_days(new2_df, set_date,ndays,extent):
    last_n_days = get_filtered_df(new2_df, set_date, ndays)
    vol_rise_in_last_n_days = whether_trade_vol_rise_in_last_n_days(last_n_days,extent=extent)
    vol_rise_in_last_n_days = vol_rise_in_last_n_days.loc[vol_rise_in_last_n_days]
    vol_rise_in_last_n_days = list(vol_rise_in_last_n_days.index) # is a list
    set5 = set(vol_rise_in_last_n_days)
    return set5
    

def today_turnover_condition(today, turnoverRate):
    turnover_larger_than = whether_turnover_rate_larger_than(today,turnoverRate)
    turnover_larger_than = turnover_larger_than[turnover_larger_than]
    set6 = set(turnover_larger_than.index)
    return set6


def get_n_days_latter(df,set_date,n_days):
    # Convert set_date to the desired format
    set_date = pd.to_datetime(set_date, format='%Y%m%d')
    # Get the date that is 30 business days before set_date
    future_day = set_date + BDay(n_days)
    filtered_df = df[df['日期'] == future_day]
    return filtered_df

def get_portfolio_return(df,set_date,scode_list,n_days):
    # buy at close price
    today = get_today(df, set_date)
    selected_scode = today[today['股票代码'].isin(scode_list)]
    buy_at = 100*selected_scode['收盘'].sum()
    # sell n days latter
    future_n_days = get_n_days_latter(df,set_date,n_days)
    selected_scode = future_n_days[future_n_days['股票代码'].isin(scode_list)]
    sell_at = 100*selected_scode['收盘'].sum()
    return (sell_at/buy_at-1)

In [2]:
# 合并dataframe
##############################
# 合并分时数据
##############################
import pickle
import efinance as ef
import pandas as pd
import numpy as np


today_df = ef.stock.get_realtime_quotes()
today_df = today_df.drop(['量比', '动态市盈率','总市值','流通市值','行情ID','市场类型','更新时间','昨日收盘'], axis=1)
today_df = today_df.rename(columns={'最新交易日': '日期'})
today_df = today_df.rename(columns={'今开': '开盘'})
today_df = today_df.rename(columns={'最新价': '收盘'})
today_df['日期'] = pd.to_datetime(today_df['日期'], format='%Y-%m-%d')
today_df.replace('-', float('nan'), inplace=True)
today_df['开盘'] = today_df['开盘'].astype(float)
today_df['最高'] = today_df['最高'].astype(float)
today_df['最低'] = today_df['最低'].astype(float)
today_df['收盘'] = today_df['收盘'].astype(float)
today_df['成交量'] = today_df['成交量'].astype(float)
today_df['成交额'] = today_df['成交额'].astype(float)
today_df['涨跌幅'] = today_df['涨跌幅'].astype(float)
today_df['涨跌额'] = today_df['涨跌额'].astype(float)
today_df['换手率'] = today_df['换手率'].astype(float)

with open('new_total_df.pkl', 'rb') as file:
    total_df = pickle.load(file)
    
new_total_df = pd.concat([total_df,today_df],axis=0)


def get_log_growth(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['收盘'] = pd.to_numeric(df['收盘'], errors='coerce')

        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期收盘'] = df.groupby('股票代码')['收盘'].shift(1)

        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['收盘涨幅对数'] = np.log(df['收盘'] / df['滞后一期收盘'])
        return df
    
def get_volume_change(df):
    with np.errstate(divide='ignore'):
        # Sort the dataframe by 'Scode' and 'Date' in ascending order
        df.sort_values(['股票代码', '日期'], inplace=True)
        df['成交量'] = pd.to_numeric(df['成交量'], errors='coerce')
        # Create the 'Lag_close' column by shifting the 'Close' column by one period
        df['滞后一期成交量'] = df.groupby('股票代码')['成交量'].shift(1)
        df['滞后一期成交量'] = df['滞后一期成交量'].replace(0, np.nan)
        # Calculate the percentage change in trade volume
        df['成交量变化百分比'] = (df['成交量'] - df['滞后一期成交量']) / df['滞后一期成交量'] * 100
        # Calculate the 'log_growth' column using the formula ln('Close'/'Lag_close')
        df['成交量涨幅对数'] = np.log(df['成交量'] / df['滞后一期成交量'])
        return df

def get_volatility(df):
    with np.errstate(divide='ignore'):
        df['最高'] = pd.to_numeric(df['最高'], errors='coerce')
        df['最低'] = pd.to_numeric(df['最低'], errors='coerce')
        df['振幅'] = abs((df['最高'] / df['滞后一期收盘']) - (df['最低'] / df['滞后一期收盘']))
        return df
    
    
def get_pillar_height(df):
    with np.errstate(divide='ignore'):
        df['开盘'] = pd.to_numeric(df['开盘'], errors='coerce')
        df['柱高'] = abs((df['开盘'] / df['滞后一期收盘']) - (df['收盘'] / df['滞后一期收盘']))
        return df

def get_line_height(df):
    with np.errstate(divide='ignore'):
        df['上影线'] = df['最高'] - df['收盘']
        return df
    
new_total_df = get_log_growth(new_total_df)
new_total_df = get_volume_change(new_total_df)
new_total_df = get_volatility(new_total_df)
new_total_df = get_pillar_height(new_total_df)
new_total_df = get_line_height(new_total_df)
new_total_df

,股票名称,股票代码,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,滞后一期收盘,收盘涨幅对数,滞后一期成交量,成交量变化百分比,成交量涨幅对数,柱高,上影线
0,平安银行,000001,2023-01-03,12.92,13.49,13.57,12.77,2194128.0,2.971547e+09,NaN,4.74,0.61,1.13,NaN,NaN,NaN,NaN,NaN,NaN,0.08
1,平安银行,000001,2023-01-04,13.43,14.04,14.14,13.35,2189683.0,3.110729e+09,0.058562,4.08,0.55,1.13,13.49,0.039962,2194128.0,-0.202586,-0.002028,0.045219,0.10
2,平安银行,000001,2023-01-05,14.12,14.20,14.46,14.09,1665425.0,2.417272e+09,0.026353,1.14,0.16,0.86,14.04,0.011332,2189683.0,-23.942187,-0.273676,0.005698,0.26
3,平安银行,000001,2023-01-06,14.22,14.34,14.44,14.20,1195745.0,1.747915e+09,0.016901,0.99,0.14,0.62,14.20,0.009811,1665425.0,-28.201810,-0.331311,0.008451,0.10
4,平安银行,000001,2023-01-09,14.47,14.52,14.60,14.24,1057659.0,1.561368e+09,0.025105,1.26,0.18,0.55,14.34,0.012474,1195745.0,-11.548114,-0.122711,0.003487,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,美心翼申,873833,2023-12-13,13.90,14.54,14.59,13.81,27167.0,3.909513e+07,0.055874,4.15,0.58,10.25,13.96,0.040707,14487.0,87.526748,0.628751,0.045845,0.05
32,美心翼申,873833,2023-12-14,14.64,15.30,15.31,14.35,44109.0,6.573880e+07,0.066025,5.23,0.76,16.64,14.54,0.050949,27167.0,62.362425,0.484661,0.045392,0.01
33,美心翼申,873833,2023-12-15,15.08,15.26,15.92,14.71,30293.0,4.617363e+07,0.079085,-0.26,-0.04,11.43,15.30,-0.002618,44109.0,-31.322406,-0.375747,0.011765,0.66
34,美心翼申,873833,2023-12-18,15.22,15.79,15.88,15.02,32376.0,5.025268e+07,0.056356,3.47,0.53,12.21,15.26,0.034142,30293.0,6.876176,0.066501,0.037353,0.09


In [16]:
######################   当日策略 ######################

import warnings
warnings.filterwarnings('ignore')

import time
import pandas as pd
# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
# Assuming you have the start and end tactic dates as strings

# 
start_tactic_date = '20231205' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  str(input('输入今天的日期：（例：20231130）'))
# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-01','2023-01-02','2023-01-03','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']
Dragon_boat = ['2023-6-22']
all_holiday = national_holiday_list+May_first+Qingming+Spring_festival+Dragon_boat


# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []

nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
result_df = pd.DataFrame(columns=['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量','今日涨幅','近三日涨幅']).set_index('股票代码')

#要点一： MA5>MA10>MA20
#要点二：
x1 = 0.5
x2 = 8
a1 = 102
a2 , a3 = 98, 101
a4 = 3
a5 = 3
a6 = 3

for index, day in enumerate(working_days):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 40)
    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9 = set(), set(), set(), set(), set(), set(), set(), set(), set()
    # MA5>MA10>MA20
    slide_minus5to0 = get_filtered_df(slide_df, working_days[index], 5) #  day - BDay(prek)
    slide_minus10to0 = get_filtered_df(slide_df, working_days[index], 10) #  day - BDay(prek)
    slide_minus20to0 = get_filtered_df(slide_df, working_days[index], 20) #  day - BDay(prek)
    MA5 = pd.DataFrame(get_average_price(slide_minus5to0))
    MA5.columns = ['MA5']
    MA10 = pd.DataFrame(get_average_price(slide_minus10to0))
    MA10.columns = ['MA10']
    MA20 = pd.DataFrame(get_average_price(slide_minus20to0))
    MA20.columns = ['MA20']
    result = pd.merge(today, MA5, on='股票代码')
    result = pd.merge(result, MA10, on='股票代码')
    result = pd.merge(result, MA20, on='股票代码')
    set1_1 = set(result[result['MA5']>result['MA10']]['股票代码'])
    set1_2 = set(result[result['MA10']>result['MA20']]['股票代码'])
    set1 = set1_1 & set1_2
    if set1:
        set2_1 = set(result[result['收盘']/result['MA5']>=100/100]['股票代码'])
        set2_2 = set(result[result['收盘']/result['MA5']<= a1/100]['股票代码'])
        set2 = set2_1 & set2_2
        set2 = set1.intersection(set2)
    if set2:
        set3_1 = set(result[result['最低']/result['MA5']>a2/100]['股票代码'])
        set3_2 = set(result[result['最低']/result['MA5']<a3/100]['股票代码'])
        set3 = set3_1 & set3_2
        set3 = set2.intersection(set3)
    if set3:
        ML5 = pd.DataFrame(get_average_volatility(slide_minus5to0))
        ML5.columns = ['ML5']
        result = pd.merge(today, ML5, on='股票代码')
        set4_1 = set(result[(result['ML5']/result['柱高'])>x2]['股票代码'])
        set4_2 = set(result[result['柱高']<x1/100]['股票代码'])
        set4_3 = set(result[result['ML5']>=5/100]['股票代码'])
        set4 = set4_1 | set4_2
        set4 = set3.intersection(set4).intersection(set4_3)
    if set4:
        # 过去5天至少有一天涨幅大于3
        slide_minus6to1 = get_filtered_df(slide_df, working_days[index-1], 5) #  day - BDay(prek)
        result_set = slide_minus6to1.groupby('股票代码')['涨跌幅'].apply(lambda x: (x > a4).any())
        set5 = set(result_set[result_set].index)
        set5 = set4.intersection(set5)
    if set5:
        # 过去5天至少有一天涨幅大于3
        slide_minus2to2 = get_filtered_df(slide_df, working_days[index-2], 1) #  day - BDay(prek)
        result_set1 = slide_minus2to2[slide_minus2to2['振幅']/slide_minus2to2['上影线']<a5]
        slide_minus1to1 = get_filtered_df(slide_df, working_days[index-1], 1) #  day - BDay(prek)
        result_set2 = slide_minus1to1[slide_minus1to1['振幅']/slide_minus1to1['上影线']<a6]
        set6_1 = set(result_set1['股票代码'])
        set6_2 = set(result_set2['股票代码'])
        set6 = set6_1 & set6_2
        set6 = set5.intersection(set6)
        
    selected_codes = {x for x in set6 if x.startswith(('00', '30', '60'))}
        
    if selected_codes:
        day_list.append(day)
        code_list.append(selected_codes)
        company_list = today[['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量']].set_index('股票代码') 
        company_list = company_list.loc[company_list.index.isin(selected_codes)]       
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange],axis=1)
        result_df = pd.concat([result_df,result_series],axis=0)
        print('Selected codes are {}'.format(selected_codes))
    else:
        print('No stock is selected')
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time: ", execution_time, " seconds")
    print('\n')
    
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
# if return_rate_total:
#     average_return = sum(return_rate_total) / len(return_rate_total)
#     print('Average return: {}'.format(average_return))
#     portfolio_dict = {
#         'Date':day_list,
#         'Scode':code_list,
#         'return_rate':return_rate_total
#     }
#     portfolio_df = pd.DataFrame(portfolio_dict)
#     portfolio_df.to_excel('portfolio_return.xlsx')
result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
result_df.to_excel('select_df_tactic3.xlsx')
# else:
#     print('No stock selected')

输入今天的日期：（例：20231130）20231218
Selecting stocks at 2023-12-05 00:00:00
No stock is selected
Execution time:  0.13630414009094238  seconds


Selecting stocks at 2023-12-06 00:00:00
No stock is selected
Execution time:  0.7882249355316162  seconds


Selecting stocks at 2023-12-07 00:00:00
Selected codes are {'300133', '000626', '300043', '600692', '002174', '002659', '300418', '300364', '600463', '300984', '000985', '000716', '002366', '300148'}
Execution time:  0.8000116348266602  seconds


Selecting stocks at 2023-12-08 00:00:00
Selected codes are {'603213', '601059'}
Execution time:  0.7880916595458984  seconds


Selecting stocks at 2023-12-11 00:00:00
Selected codes are {'300011', '000922'}
Execution time:  0.780087947845459  seconds


Selecting stocks at 2023-12-12 00:00:00
Selected codes are {'300862', '300426', '002542', '301321', '603477', '300182', '300081', '603861', '301387', '600229', '300299', '300027', '002480', '300413', '600088', '300058', '603803'}
Execution time:  0.79809

In [70]:
######################   新的回测   ######################


import time

# set_date = '20230926'
# day = pd.to_datetime(set_date, format='%Y%m%d')
    

import pandas as pd

# Assuming you have the start and end tactic dates as strings
start_tactic_date = '20221201' #str(input('输入测试开始时间：（例：20231001）'))
end_tactic_date =  '20231127' #str(input('输入测试结束时间：（例：20231031）'))
# Convert the tactic dates to datetime objects
start_date = pd.to_datetime(start_tactic_date, format='%Y%m%d')
end_date = pd.to_datetime(end_tactic_date, format='%Y%m%d')
# Generate the series of working days
working_days = pd.bdate_range(start=start_date, end=end_date)
national_holiday_list = ['2023-09-29','2023-10-02','2023-10-03','2023-10-04','2023-10-05','2023-10-06']
May_first = ['2023-05-01','2023-05-02','2023-05-03']
Qingming = ['2023-04-05']
Spring_festival = ['2023-01-01','2023-01-02','2023-01-03','2023-01-23','2023-01-24','2023-01-25','2023-01-26','2023-01-27']
Dragon_boat = ['2023-6-22']
all_holiday = national_holiday_list+May_first+Qingming+Spring_festival+Dragon_boat

for day in all_holiday:
    if day in working_days:
        working_days = working_days.drop(day)

  
# 涨幅限制用于切片，涨幅天数是最大限制。 

start_time_original = time.time()
# portfolio args
return_rate_total = []
day_list = []
code_list = []


nday_sold = 1 #int(input("请设置在购买后第几天卖出："))
result_df = pd.DataFrame(columns=['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量','今日涨幅','近三日涨幅','D+1涨幅','D+2涨幅','D+3涨幅','D+4涨幅','D+5涨幅','D+6涨幅','D+7涨幅','D+8涨幅','D+9涨幅','D+10涨幅','D+11涨幅','D+12涨幅','D+13涨幅','D+14涨幅','D+15涨幅','D+16涨幅','D+17涨幅','D+18涨幅','D+19涨幅','D+20涨幅']).set_index('股票代码')


#要点一： MA5>MA10>MA20
#要点二：
x1 = 0.5
x2 = 8
a1 = 102
a2 , a3 = 98, 101
a4 = 3

for index, day in enumerate(working_days):
    start_time = time.time()
    slide_df = get_filtered_df(new_total_df, working_days[index], 40)
    print('Selecting stocks at {}'.format(day))
    # 计算今天的情况
    today = get_today(slide_df, day)
    today_vol = get_average_vol(today)
    today_growth = get_growth(today)
    se1, set2, set3, set4, set5, set6, set7, set8, set9 = set(), set(), set(), set(), set(), set(), set(), set(), set()
    # MA5>MA10>MA20
    slide_minus5to0 = get_filtered_df(slide_df, working_days[index], 5) #  day - BDay(prek)
    slide_minus10to0 = get_filtered_df(slide_df, working_days[index], 10) #  day - BDay(prek)
    slide_minus20to0 = get_filtered_df(slide_df, working_days[index], 20) #  day - BDay(prek)
    MA5 = pd.DataFrame(get_average_price(slide_minus5to0))
    MA5.columns = ['MA5']
    MA10 = pd.DataFrame(get_average_price(slide_minus10to0))
    MA10.columns = ['MA10']
    MA20 = pd.DataFrame(get_average_price(slide_minus20to0))
    MA20.columns = ['MA20']
    result = pd.merge(today, MA5, on='股票代码')
    result = pd.merge(result, MA10, on='股票代码')
    result = pd.merge(result, MA20, on='股票代码')
    set1_1 = set(result[result['MA5']>result['MA10']]['股票代码'])
    set1_2 = set(result[result['MA10']>result['MA20']]['股票代码'])
    set1 = set1_1 & set1_2
    if set1:
        set2_1 = set(result[result['收盘']/result['MA5']>=100/100]['股票代码'])
        set2_2 = set(result[result['收盘']/result['MA5']<= a1/100]['股票代码'])
        set2 = set2_1 & set2_2
        set2 = set1.intersection(set2)
    if set2:
        set3_1 = set(result[result['最低']/result['MA5']>a2/100]['股票代码'])
        set3_2 = set(result[result['最低']/result['MA5']<a3/100]['股票代码'])
        set3 = set3_1 & set3_2
        set3 = set2.intersection(set3)
    if set3:
        ML5 = pd.DataFrame(get_average_volatility(slide_minus5to0))
        ML5.columns = ['ML5']
        result = pd.merge(today, ML5, on='股票代码')
        set4_1 = set(result[(result['ML5']/result['柱高'])>x2]['股票代码'])
        set4_2 = set(result[result['柱高']<x1/100]['股票代码'])
        set4_3 = set(result[result['ML5']>=5/100]['股票代码'])
        set4 = set4_1 | set4_2
        set4 = set3.intersection(set4).intersection(set4_3)
    if set4:
        # 过去5天至少有一天涨幅大于3
        slide_minus6to1 = get_filtered_df(slide_df, working_days[index-1], 5) #  day - BDay(prek)
        result_set = slide_minus6to1.groupby('股票代码')['涨跌幅'].apply(lambda x: (x > a4).any())
        set5 = set(result_set[result_set].index)
        set5 = set4.intersection(set5)
    if set5:
        # 过去5天至少有一天涨幅大于3
        slide_minus2to2 = get_filtered_df(slide_df, working_days[index-2], 1) #  day - BDay(prek)
        result_set1 = slide_minus2to2[slide_minus2to2['振幅']/slide_minus2to2['上影线']<a5]
        slide_minus1to1 = get_filtered_df(slide_df, working_days[index-1], 1) #  day - BDay(prek)
        result_set2 = slide_minus1to1[slide_minus1to1['振幅']/slide_minus1to1['上影线']<a6]
        set6_1 = set(result_set1['股票代码'])
        set6_2 = set(result_set2['股票代码'])
        set6 = set6_1 & set6_2
        set6 = set5.intersection(set6)
        
    selected_codes = {x for x in set6 if x.startswith(('00', '30', '60'))}
        
    if selected_codes:
        return_df = get_filtered_df(new_total_df,working_days[index+20],25) # day+BDay(20)
        return_rate = get_portfolio_return(new_total_df,day,selected_codes,nday_sold)
        day_list.append(day)
        code_list.append(selected_codes)
        return_rate_total.append(return_rate)   
        company_list = today[['股票代码','股票名称','日期','开盘','收盘','换手率','最高','成交量']].set_index('股票代码')
        company_list = company_list.loc[company_list.index.isin(selected_codes)]       
        today_pchange = pd.DataFrame(today_growth).rename(columns={'收盘涨幅对数': '今日涨幅'})
        today_pchange = today_pchange.loc[today_pchange.index.isin(selected_codes)]
        recent3daychange = pd.DataFrame(get_growth(get_filtered_df(slide_df, working_days[index], 3))).rename(columns={'收盘涨幅对数': '近三日涨幅'})
        recent3daychange = recent3daychange.loc[recent3daychange.index.isin(selected_codes)]
        D1_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+1],1))).rename(columns={'收盘涨幅对数': 'D+1涨幅'}) # day+BDay(1)
        D1_pchange = D1_pchange.loc[D1_pchange.index.isin(selected_codes)]
        D2_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+2],2))).rename(columns={'收盘涨幅对数': 'D+2涨幅'}) # day+BDay(2)
        D2_pchange = D2_pchange.loc[D2_pchange.index.isin(selected_codes)]
        D3_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df,working_days[index+3] ,3))).rename(columns={'收盘涨幅对数': 'D+3涨幅'}) # day+BDay(3)
        D3_pchange = D3_pchange.loc[D3_pchange.index.isin(selected_codes)]
        D4_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+4],4))).rename(columns={'收盘涨幅对数': 'D+4涨幅'}) # day+BDay(4)
        D4_pchange = D4_pchange.loc[D4_pchange.index.isin(selected_codes)]
        D5_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+5] ,5))).rename(columns={'收盘涨幅对数': 'D+5涨幅'}) # day+BDay(5)
        D5_pchange = D5_pchange.loc[D5_pchange.index.isin(selected_codes)]
        D6_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+6] ,6))).rename(columns={'收盘涨幅对数': 'D+6涨幅'}) # day+BDay(6)
        D6_pchange = D6_pchange.loc[D6_pchange.index.isin(selected_codes)]
        D7_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+7] ,7))).rename(columns={'收盘涨幅对数': 'D+7涨幅'}) # day+BDay(7)
        D7_pchange = D7_pchange.loc[D7_pchange.index.isin(selected_codes)]
        D8_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+8] ,8))).rename(columns={'收盘涨幅对数': 'D+8涨幅'}) # day+BDay(8)
        D8_pchange = D8_pchange.loc[D8_pchange.index.isin(selected_codes)]
        D9_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+9] ,9))).rename(columns={'收盘涨幅对数': 'D+9涨幅'}) # day+BDay(9)
        D9_pchange = D9_pchange.loc[D9_pchange.index.isin(selected_codes)]
        D10_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+10] ,10))).rename(columns={'收盘涨幅对数': 'D+10涨幅'}) # day+BDay(10)
        D10_pchange = D10_pchange.loc[D10_pchange.index.isin(selected_codes)]
        D11_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+11] ,11))).rename(columns={'收盘涨幅对数': 'D+11涨幅'}) # day+BDay(11)
        D11_pchange = D11_pchange.loc[D11_pchange.index.isin(selected_codes)]
        D12_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+12] ,12))).rename(columns={'收盘涨幅对数': 'D+12涨幅'}) # day+BDay(12)
        D12_pchange = D12_pchange.loc[D12_pchange.index.isin(selected_codes)]
        D13_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+13] ,13))).rename(columns={'收盘涨幅对数': 'D+13涨幅'}) # day+BDay(13)
        D13_pchange = D13_pchange.loc[D13_pchange.index.isin(selected_codes)]
        D14_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+14] ,14))).rename(columns={'收盘涨幅对数': 'D+14涨幅'}) # day+BDay(14)
        D14_pchange = D14_pchange.loc[D14_pchange.index.isin(selected_codes)]
        D15_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+15] ,15))).rename(columns={'收盘涨幅对数': 'D+15涨幅'}) # day+BDay(15)
        D15_pchange = D15_pchange.loc[D15_pchange.index.isin(selected_codes)]
        D16_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+16] ,16))).rename(columns={'收盘涨幅对数': 'D+16涨幅'}) # day+BDay(16)
        D16_pchange = D16_pchange.loc[D16_pchange.index.isin(selected_codes)]
        D17_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+17] ,17))).rename(columns={'收盘涨幅对数': 'D+17涨幅'}) # day+BDay(17)
        D17_pchange = D17_pchange.loc[D17_pchange.index.isin(selected_codes)]
        D18_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+18] ,18))).rename(columns={'收盘涨幅对数': 'D+18涨幅'}) # day+BDay(18)
        D18_pchange = D18_pchange.loc[D18_pchange.index.isin(selected_codes)]
        D19_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+19] ,19))).rename(columns={'收盘涨幅对数': 'D+19涨幅'}) # day+BDay(19)
        D19_pchange = D19_pchange.loc[D19_pchange.index.isin(selected_codes)]
        D20_pchange = pd.DataFrame(get_growth(get_filtered_df(return_df, working_days[index+20] ,20))).rename(columns={'收盘涨幅对数': 'D+20涨幅'}) # day+BDay(20)
        D20_pchange = D20_pchange.loc[D20_pchange.index.isin(selected_codes)]
        result_series = pd.concat([company_list,today_pchange,recent3daychange, D1_pchange,D2_pchange,D3_pchange,D4_pchange,D5_pchange,D6_pchange,D7_pchange,D8_pchange,D9_pchange,D10_pchange,D11_pchange,D12_pchange,D13_pchange,D14_pchange,D15_pchange,D16_pchange,D17_pchange,D18_pchange,D19_pchange,D20_pchange],axis=1)
        
        result_df = pd.concat([result_df,result_series],axis=0)
        print('Selected codes are {}'.format(selected_codes))
        print('Getting return rate of {}'.format(return_rate))
    else:
        print('No stock is selected')
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time: ", execution_time, " seconds")
    print('\n')
    
end_time_all = time.time()
execution_time_all = end_time_all-start_time_original

print('Finish execution, using {} seconds'.format(execution_time_all))
if return_rate_total:
    average_return = sum(return_rate_total) / len(return_rate_total)
    print('Average return: {}'.format(average_return))
    portfolio_dict = {
        'Date':day_list,
        'Scode':code_list,
        'return_rate':return_rate_total
    }
    portfolio_df = pd.DataFrame(portfolio_dict)
    portfolio_df.to_excel('portfolio_return.xlsx')
    result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
    result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
    result_df['Ntimes'] = result_df['Ntimes'].astype(float)
    result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
    result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
    result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
    result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
    result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
    result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
    result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
    result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
    result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
    result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
    result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
    result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
    result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
    result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
    result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
    result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
    result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
    result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
    result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
    result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
    result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
    result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
    result_df.to_excel('result_df.xlsx')
else:
    print('No stock selected')

Selecting stocks at 2022-12-01 00:00:00
Selected codes are {'000042', '000692', '605289', '002853', '002786', '600340', '002537', '003028', '600153', '003000', '000899', '002372', '600941', '001255', '002398', '603833', '001322', '600333', '002162', '688676', '002231', '600383'}
Getting return rate of 0.0023449552607219815
Execution time:  2.1400835514068604  seconds


Selecting stocks at 2022-12-02 00:00:00
Selected codes are {'600358', '000032', '000716', '002520', '002647', '300356', '002659', '002786', '300084', '300246', '301168', '301227', '000899', '300551', '300986', '002175', '002159', '600698', '600624', '600693', '300965', '002790', '301349', '002251', '300338', '688317', '000025'}
Getting return rate of -0.006577469104279476
Execution time:  2.1997034549713135  seconds


Selecting stocks at 2022-12-05 00:00:00
Selected codes are {'600833', '000002', '000046', '603230', '300767', '002775', '300857', '000069', '600708', '002703', '002523', '600006', '600418', '002090', '60083

KeyboardInterrupt: 

In [39]:
# result_df['今日涨幅'] = result_df['今日涨幅'].astype(float)
# result_df['近三日涨幅'] = result_df['近三日涨幅'].astype(float)
# result_df['D+1涨幅'] = result_df['D+1涨幅'].astype(float)
# result_df['D+2涨幅'] = result_df['D+2涨幅'].astype(float)
# result_df['D+3涨幅'] = result_df['D+3涨幅'].astype(float)
# result_df['D+4涨幅'] = result_df['D+4涨幅'].astype(float)
# result_df['D+5涨幅'] = result_df['D+5涨幅'].astype(float)
# result_df['D+6涨幅'] = result_df['D+6涨幅'].astype(float)
# result_df['D+7涨幅'] = result_df['D+7涨幅'].astype(float)
# result_df['D+8涨幅'] = result_df['D+8涨幅'].astype(float)
# result_df['D+9涨幅'] = result_df['D+9涨幅'].astype(float)
# result_df['D+10涨幅'] = result_df['D+10涨幅'].astype(float)
# result_df['D+11涨幅'] = result_df['D+11涨幅'].astype(float)
# result_df['D+12涨幅'] = result_df['D+12涨幅'].astype(float)
# result_df['D+13涨幅'] = result_df['D+13涨幅'].astype(float)
# result_df['D+14涨幅'] = result_df['D+14涨幅'].astype(float)
# result_df['D+15涨幅'] = result_df['D+15涨幅'].astype(float)
# result_df['D+16涨幅'] = result_df['D+16涨幅'].astype(float)
# result_df['D+17涨幅'] = result_df['D+17涨幅'].astype(float)
# result_df['D+18涨幅'] = result_df['D+18涨幅'].astype(float)
# result_df['D+19涨幅'] = result_df['D+19涨幅'].astype(float)
# result_df['D+20涨幅'] = result_df['D+20涨幅'].astype(float)
# result_df['今日涨幅'] = result_df['今日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['近三日涨幅'] = result_df['近三日涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+1涨幅'] = result_df['D+1涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+2涨幅'] = result_df['D+2涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+3涨幅'] = result_df['D+3涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+4涨幅'] = result_df['D+4涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+5涨幅'] = result_df['D+5涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+6涨幅'] = result_df['D+6涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+7涨幅'] = result_df['D+7涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+8涨幅'] = result_df['D+8涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+9涨幅'] = result_df['D+9涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+10涨幅'] = result_df['D+10涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+11涨幅'] = result_df['D+11涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+12涨幅'] = result_df['D+12涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+13涨幅'] = result_df['D+13涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+14涨幅'] = result_df['D+14涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+15涨幅'] = result_df['D+15涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+16涨幅'] = result_df['D+16涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+17涨幅'] = result_df['D+17涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+18涨幅'] = result_df['D+18涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+19涨幅'] = result_df['D+19涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['D+20涨幅'] = result_df['D+20涨幅'].map(lambda x: "{:.2f}".format(x * 100))
# result_df['日期'] = pd.to_datetime(result_df['日期'], format='%Y-%m-%d')
# result_df.to_excel('result_df.xlsx')

<ipython-input-39-565c79222f58>:46: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel('result_df.xlsx')
